In [12]:
import lal

def make_list_hv(file):
    list_hv = []
    with open(file, 'r') as f:
        data = []
        for line in f:
            if line[0] == '#':
                pass
            elif line == '\n':
                hv = list(map(lambda l: int(l.split('\t')[6]), data))
                list_hv.append(hv)
                data = []
            else:
                data.append(line[:-1])
        if len(data) > 0:
            hv = list(map(lambda l: int(l.split('\t')[6]), data))
            list_hv.append(hv)
    return list_hv

list_hv = make_list_hv("P21CE2E2.conllu")
print(list_hv)

hv_hid = {}
total_list_mhd = []

[[4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14], [4, 4, 4, 0, 4, 4, 8, 6, 8, 9, 4, 14, 14, 11, 16, 14, 14, 17], [4, 4, 4, 0, 4, 4, 8, 6, 8, 9, 4, 14, 14, 11, 16, 14, 14, 17], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14, 16, 17, 18, 23, 23, 21, 0], [4, 4, 4, 0, 4, 4, 8, 6, 8, 9, 4, 14, 14, 11, 16, 14, 14, 17, 18, 0, 0], [4, 4, 4, 0, 4, 4, 8, 6, 8, 9, 4, 14, 14, 11, 16, 14, 14, 17, 18, 0, 0], [4, 4, 4, 0, 4, 4, 8, 6, 8, 9, 4, 14, 14, 11, 16, 14, 14, 17, 18, 0, 0], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14, 16, 17, 18, 23, 23, 23, 0, 23, 26, 23, 26], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14, 16, 17, 18, 23, 23, 23, 0, 23, 26, 23], [4, 4, 4, 0, 4, 4, 8, 4, 4, 9, 9, 14, 14, 9, 16, 14, 16, 17

In [15]:
head_initial_proportions = []
for i in range(len(list_hv)):
    dg = lal.graphs.from_head_vector_to_directed_graph(list_hv[i])
    ccs = lal.properties.compute_connected_components(dg)
    list_head_init = []
    for j in range(0, ccs.size()):
        if ccs[j].get_num_nodes() > 1: # error if only one node in the sentence
            rt = lal.graphs.rooted_tree(ccs[j])
            head_init = lal.linarr.head_initial(rt)
            list_head_init.append(head_init)
            head_initial_proportions.append(head_init)
        else:
            head_init = 0
            list_head_init.append(head_init)
            head_initial_proportions.append(head_init)
    hv_hid[i] = list_head_init

print(head_initial_proportions)

[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9411764705882353, 0.9411764705882353, 0.9444444444444444, 0.6666666666666666, 0.9444444444444444, 0, 0, 0.9444444444444444, 0, 0, 0.9444444444444444, 0, 0, 0.9444444444444444, 0.8571428571428571, 0.9444444444444444, 0.8333333333333334, 0.9444444444444444, 0.8333333333333334, 0.9444444444444444, 0, 0, 0, 0.9444444444444444, 0.8, 0.6666666666666666, 0.9444444444444444, 0.8, 0.6666666666666666, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 1.0, 0, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 0.8, 0.8, 1.0, 0.9444444444444444, 1.0, 0, 1.0, 0.9444444444444444, 1.0, 0, 1.0]


In [16]:
import plotly.express as px

fig = px.line(y=head_initial_proportions, title='Proportion of head initial dependencies', labels={'y': 'Proportion', 'x': 'Root'})

fig.show()

In [10]:
import pandas as pd
import plotly.graph_objects as go
flat_mhd_values = []
sentence_boundaries = []
current_index = 0

for k, values in hv_hid.items():
    flat_mhd_values.extend(values)
    current_index += len(values)
    sentence_boundaries.append(current_index - 1)  # Subtract 1 because index is 0-based

# Create a Plotly figure
fig = go.Figure()

# Add a line trace for mhd_values
fig.add_trace(go.Scatter(x=list(range(len(flat_mhd_values))), y=flat_mhd_values, mode='lines', name='MHD Values'))

# Add vertical lines at sentence boundaries
for boundary in sentence_boundaries[:-1]:  # Skip the last boundary
    fig.add_vline(x=boundary + 0.5, line_width=2, line_dash="dash", line_color="grey")

# Update layout
fig.update_layout(title='Proportion of head initial dependencies with burst boundaries', xaxis_title='Roots', yaxis_title='Proportion', xaxis=dict(range=[0, 83]))

# Show plot
fig.show()